### Quest: Divide anime into groups and find a model to classify groups, predict anime rating.

> To-do list:
>1. Analitics
2. Preprosessing
3. Clustering
4. Classification
5. Regression

In [21]:
# ! pip install seaborn

In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as ptl
from collections import Counter

# Unpack the archive with dataset

In [18]:
df = pd.read_csv('./data/anime.csv')
df.head()

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009.0,2010.0,Spring,The foundation of alchemy is based on the law ...,['Bones'],"['Action', 'Adventure', 'Drama', 'Fantasy', 'M...","['Animal Abuse', 'Mature Themes', 'Violence', ...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016.0,2016.0,NaN,Mitsuha and Taki are two total strangers livin...,['CoMix Wave Films'],"['Drama', 'Romance', 'Body Swapping', 'Gender ...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016.0,2016.0,NaN,"After transferring into a new school, a deaf g...",['Kyoto Animation'],"['Drama', 'Shounen', 'Disability', 'Melancholy...","['Bullying', 'Mature Themes', 'Suicide']",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016.0,2016.0,Fall,"Picking up where the second season ended, the ...",['Production I.G'],"['Shounen', 'Sports', 'Animeism', 'School Club...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019.0,2019.0,Spring,The battle to retake Wall Maria begins now! Wi...,['Wit Studio'],"['Action', 'Fantasy', 'Horror', 'Shounen', 'Da...","['Cannibalism', 'Explicit Violence']",21308.0,3217,7864,174,4.650,15789.0


## Analytics

### Anotation on all dataset

In [91]:
df.shape

(14578, 18)

dataset has 14578 columns and 18 rows

#### Type datas in columns

In [93]:
df.dtypes

title            object
mediaType        object
eps             float64
duration        float64
ongoing            bool
startYr         float64
finishYr        float64
sznOfRelease     object
description      object
studios          object
tags             object
contentWarn      object
watched         float64
watching          int64
wantWatch         int64
dropped           int64
rating          float64
votes           float64
dtype: object

we have columns whith number, bool type data and object (heap types) 

In [90]:
df.isnull().sum(axis=0)

title               0
mediaType          68
eps               359
duration         5441
ongoing             0
startYr           222
finishYr          444
sznOfRelease    10811
description      6405
studios             0
tags                0
contentWarn         0
watched           222
watching            0
wantWatch           0
dropped             0
rating           2471
votes            2459
dtype: int64

in most columns we have null data

but null data in column sznOfRelease > 1/2 count rows in all dataset

### Description columns

* **title**: title anime 
* **mediaType**: display format anime (TV/Movie)
* **eps**: count episodes
* **duration**: duration episodes
* **ongoing**: if unfinished anime (true), else (false)
* **startYr**: year start filming anime
* **finishYr**: year finish filming anime
* **sznOfRelease**: season of realease amine
* **description**: desc anime
* **studios**: studios filming anime
* **tags**: anime related tags
* **contentWarn**: content tags
* **watched**: watched (status)
* **watching**: watching (status)
* **wantWatch**: want watch (status)
* **dropped**: dropped (status)
* **rating**: rating
* **votes**: votes

### Analysis number columns

In [97]:
df.describe()

,eps,duration,startYr,finishYr,watched,watching,wantWatch,dropped,rating,votes
count,14219.000000,9137.000000,14356.000000,14134.000000,14356.000000,14578.000000,14578.000000,14578.000000,12107.000000,12119.000000
mean,13.501231,23.465142,2005.457788,2005.515919,2408.043396,213.026684,1021.729112,125.963026,2.948697,2085.787771
std,62.262185,30.777048,14.707105,14.656509,7168.368428,1261.707640,2145.010604,453.577348,0.827642,5946.283685
min,1.000000,1.000000,1907.000000,1907.000000,0.000000,0.000000,0.000000,0.000000,0.844000,10.000000
25%,1.000000,3.000000,2000.000000,2000.000000,25.000000,1.000000,24.000000,1.000000,2.303500,34.000000
50%,1.000000,8.000000,2010.000000,2010.000000,165.000000,7.000000,175.000000,7.000000,2.965000,218.000000
75%,12.000000,29.000000,2016.000000,2016.000000,1469.500000,63.000000,980.000000,40.000000,3.615500,1412.500000
max,2527.000000,235.000000,2026.000000,2026.000000,161567.000000,74537.000000,28541.000000,19481.000000,4.702000,131067.000000


### Analysis object columns

In [172]:
def analysis_object_column(col):
    print('Null:', df[col].isnull().sum(), f'({round(df[col].isnull().sum()/df.shape[0]*100)}%)')
    print('------------------------------------------------------')
    unique_count = len(df[col].unique())
    print(f'Unique: {unique_count} ({round(unique_count/df.shape[0]*100)}%)')
    if (unique_count < df.shape[0] * 0.8):
        if (unique_count > 30):
            print('30 classes < and < 80%')
        else:
            print(df[col].unique())
    print('------------------------------------------------------')
    for i in range(3):
        print(f'{i})', type(df[col].iloc[i]), df[col].iloc[i])

#### title

In [173]:
analysis_object_column('title')

Null: 0 (0%)
------------------------------------------------------
Unique: 14578 (100%)
------------------------------------------------------
0) <class 'str'> Fullmetal Alchemist: Brotherhood
1) <class 'str'> your name.
2) <class 'str'> A Silent Voice


column is filled with unique values

#### mediaType

In [175]:
analysis_object_column('mediaType')

Null: 68 (0%)
------------------------------------------------------
Unique: 9 (0%)
['TV' 'Movie' 'Web' 'TV Special' 'OVA' nan 'DVD Special' 'Other'
 'Music Video']
------------------------------------------------------
0) <class 'str'> TV
1) <class 'str'> Movie
2) <class 'str'> Movie


column filled with such classes: TV, Movie, Web, TV Special, OVA, DVD Special, Other, Music Video.

row with null we connot fill

#### sznOfRelease

In [176]:
analysis_object_column('sznOfRelease')

Null: 10811 (74%)
------------------------------------------------------
Unique: 5 (0%)
['Spring' nan 'Fall' 'Winter' 'Summer']
------------------------------------------------------
0) <class 'str'> Spring
1) <class 'float'> nan
2) <class 'float'> nan


column has many null value

season of realease not relevant to the definition of a cluster

#### description

In [177]:
analysis_object_column('description')

Null: 6405 (44%)
------------------------------------------------------
Unique: 8109 (56%)
30 classes < and < 80%
------------------------------------------------------
0) <class 'str'> The foundation of alchemy is based on the law of equivalent exchange; you cannot produce something from nothing. As such, alchemy is bound by one taboo - human transmutation. Four years ago two young brothers, Edward and Alphonse Elric, broke this taboo when they tried to resurrect their dead mother. During the process Al's body disintegrated and Ed lost his leg. In a desperate attempt to prevent his brother from disappearing completely, Ed sacrificed one of his arms so he could affix Al's soul to a suit of armor. When his missing limbs are replaced by auto mail parts, Ed bears the name of the Fullmetal Alchemist - the youngest ever State Alchemist and dog of the military. Now, alongside his brother, Ed uses his status within the military to attempt to find any way that he can return their bodies back t

all values is text or null

row with null we connot fill

#### studios

In [178]:
analysis_object_column('studios')

Null: 0 (0%)
------------------------------------------------------
Unique: 864 (6%)
30 classes < and < 80%
------------------------------------------------------
0) <class 'str'> ['Bones']
1) <class 'str'> ['CoMix Wave Films']
2) <class 'str'> ['Kyoto Animation']


rows consist of string with studios name list

need split strings on studios name lists

#### tags

In [179]:
analysis_object_column('tags')

Null: 0 (0%)
------------------------------------------------------
Unique: 9580 (66%)
30 classes < and < 80%
------------------------------------------------------
0) <class 'str'> ['Action', 'Adventure', 'Drama', 'Fantasy', 'Mystery', 'Shounen', 'Conspiracy', 'Military', 'Siblings', 'Based on a Manga']
1) <class 'str'> ['Drama', 'Romance', 'Body Swapping', 'Gender Bender', 'Opposites Attract', 'School Life', 'Supernatural', 'Original Work']
2) <class 'str'> ['Drama', 'Shounen', 'Disability', 'Melancholy', 'Mental Illness', 'School Life', 'Based on a Manga']


rows consist of string with tag name list

need split strings on tag name lists

#### contentWarn

In [180]:
analysis_object_column('contentWarn')

Null: 0 (0%)
------------------------------------------------------
Unique: 178 (1%)
30 classes < and < 80%
------------------------------------------------------
0) <class 'str'> ['Animal Abuse', 'Mature Themes', 'Violence', 'Domestic Abuse']
1) <class 'str'> []
2) <class 'str'> ['Bullying', 'Mature Themes', 'Suicide']


rows consist of string with content tags list

need split strings on content tags list